# 🎭 ModalX v2 - Emotion Model Training

**All fixes applied:**
- ✅ No Kaggle needed (synthetic data)
- ✅ No Google Drive needed (local save)
- ✅ Auto-download after training

---

In [ ]:
# Install dependencies
!pip install -q torch torchaudio scikit-learn tqdm matplotlib

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

# Create save directory
SAVE_DIR = '/content/modalx_weights'
os.makedirs(SAVE_DIR, exist_ok=True)
print(f'Weights will be saved to: {SAVE_DIR}')

## Dataset (Synthetic for testing)
⚠️ Replace with real RAVDESS/TESS for competition!

In [ ]:
class EmotionDataset(Dataset):
    EMOTIONS = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise', 'calm']
    
    def __init__(self, num_samples=1000, sample_rate=16000, duration=3.0):
        self.num_samples = num_samples
        self.audio_length = int(sample_rate * duration)
        self.labels = torch.randint(0, 8, (num_samples,))
    
    def __len__(self):
        return self.num_samples
    
    def __getitem__(self, idx):
        waveform = torch.randn(self.audio_length) * 0.1
        return waveform, self.labels[idx].item()

train_dataset = EmotionDataset(1600)
val_dataset = EmotionDataset(400)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, num_workers=2)

print(f'Train: {len(train_dataset)}, Val: {len(val_dataset)}')

## Model Definition

In [ ]:
class AttentionPooling(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(embed_dim, embed_dim // 2),
            nn.Tanh(),
            nn.Linear(embed_dim // 2, 1)
        )
    
    def forward(self, x):
        weights = F.softmax(self.attention(x), dim=1)
        return torch.sum(x * weights, dim=1)


class TransformerEmotionModel(nn.Module):
    EMOTIONS = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise', 'calm']
    
    def __init__(self, n_mels=80, embed_dim=256, num_heads=4, num_layers=3, num_emotions=8, dropout=0.1, sample_rate=16000):
        super().__init__()
        
        self.mel_spec = torchaudio.transforms.MelSpectrogram(
            sample_rate=sample_rate, n_fft=400, hop_length=160, n_mels=n_mels
        )
        
        self.cnn_extractor = nn.Sequential(
            nn.Conv1d(n_mels, 128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm1d(256), nn.ReLU(), nn.MaxPool1d(2),
            nn.Conv1d(256, embed_dim, kernel_size=3, padding=1),
            nn.BatchNorm1d(embed_dim), nn.ReLU(), nn.MaxPool1d(2),
        )
        
        self.pos_encoding = nn.Parameter(torch.randn(1, 1000, embed_dim) * 0.02)
        
        self.transformer_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=embed_dim, nhead=num_heads,
                dim_feedforward=embed_dim * 4, dropout=dropout, batch_first=True
            )
            for _ in range(num_layers)
        ])
        
        self.attention_pool = AttentionPooling(embed_dim)
        
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, embed_dim // 2),
            nn.ReLU(), nn.Dropout(dropout),
            nn.Linear(embed_dim // 2, num_emotions)
        )
    
    def forward(self, waveform):
        mel = self.mel_spec(waveform)
        mel = torch.log(mel + 1e-9)
        features = self.cnn_extractor(mel)
        features = features.transpose(1, 2)
        seq_len = features.size(1)
        features = features + self.pos_encoding[:, :seq_len, :]
        for layer in self.transformer_layers:
            features = layer(features)
        pooled = self.attention_pool(features)
        return self.classifier(pooled)

model = TransformerEmotionModel().to(device)
print(f'Parameters: {sum(p.numel() for p in model.parameters()):,}')

## Training

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

num_epochs = 20
best_val_acc = 0
history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

for epoch in range(num_epochs):
    model.train()
    train_loss, train_correct, train_total = 0, 0, 0
    
    for waveforms, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        waveforms, labels = waveforms.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = model(waveforms)
        loss = criterion(logits, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
        train_correct += (logits.argmax(-1) == labels).sum().item()
        train_total += labels.size(0)
    
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    with torch.no_grad():
        for waveforms, labels in val_loader:
            waveforms, labels = waveforms.to(device), labels.to(device)
            logits = model(waveforms)
            loss = criterion(logits, labels)
            val_loss += loss.item()
            val_correct += (logits.argmax(-1) == labels).sum().item()
            val_total += labels.size(0)
    
    scheduler.step()
    train_acc = train_correct / train_total
    val_acc = val_correct / val_total
    history['train_loss'].append(train_loss / len(train_loader))
    history['val_loss'].append(val_loss / len(val_loader))
    history['train_acc'].append(train_acc)
    history['val_acc'].append(val_acc)
    
    print(f'Epoch {epoch+1}: Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}')
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), f'{SAVE_DIR}/emotion_transformer.pt')
        print(f'  Saved best model!')

print(f'Best val acc: {best_val_acc:.4f}')

## Visualize Results

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(history['train_loss'], label='Train')
axes[0].plot(history['val_loss'], label='Val')
axes[0].set_title('Loss')
axes[0].legend()
axes[1].plot(history['train_acc'], label='Train')
axes[1].plot(history['val_acc'], label='Val')
axes[1].set_title('Accuracy')
axes[1].legend()
plt.tight_layout()
plt.show()

## Download Model

In [ ]:
from google.colab import files
files.download(f'{SAVE_DIR}/emotion_transformer.pt')
print('Done! Put emotion_transformer.pt in modalx_v2/weights/')